In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [47]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
data = data.drop(columns = ['RowNumber','CustomerId','Surname']).reset_index(drop=True)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [49]:
# Load the pickle files
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

In [50]:
## Encode the categorical features
geo_encoded = onehot_encoder_geo.transform(data[['Geography']]).toarray()
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [51]:
geo_encoded = pd.DataFrame(data = geo_encoded, columns = onehot_encoder_geo.get_feature_names_out())
geo_encoded

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [52]:
data['Gender'] = label_encoder_gender.transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [53]:
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [54]:
data = pd.concat([data.drop(columns = ['Geography']),geo_encoded], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [55]:
# Dependent and Independent variables
X = data.drop(columns = ['EstimatedSalary'])
y = data['EstimatedSalary']

In [56]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=43)

In [58]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
2724,453,0,38,8,120623.21,1,1,0,0,0.0,1.0,0.0
5057,753,0,46,3,111512.75,3,1,0,1,0.0,1.0,0.0
6027,850,1,44,3,140393.65,2,0,1,0,0.0,1.0,0.0
6729,454,0,50,10,92895.56,1,1,0,1,0.0,1.0,0.0
9132,635,0,33,5,0.00,2,1,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8499,498,1,35,2,121968.11,2,0,1,0,0.0,1.0,0.0
2064,765,1,44,6,0.00,2,1,1,0,1.0,0.0,0.0
7985,696,0,27,2,96129.32,2,1,1,0,0.0,1.0,0.0
2303,714,1,45,0,124693.48,1,0,1,0,0.0,0.0,1.0


In [59]:
## Scale the data
scaler_reg = StandardScaler()
scale_columns = ['CreditScore','Age','Tenure','Balance','NumOfProducts']
X_train[scale_columns]


,CreditScore,Age,Tenure,Balance,NumOfProducts
2724,453,38,8,120623.21,1
5057,753,46,3,111512.75,3
6027,850,44,3,140393.65,2
6729,454,50,10,92895.56,1
9132,635,33,5,0.00,2
...,...,...,...,...,...
8499,498,35,2,121968.11,2
2064,765,44,6,0.00,2
7985,696,27,2,96129.32,2
2303,714,45,0,124693.48,1


In [60]:
X_train[scale_columns] = scaler_reg.fit_transform(X_train[scale_columns])
X_train


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
2724,-2.030028,0,-0.100679,1.032376,0.708724,-0.917116,1,0,0,0.0,1.0,0.0
5057,1.066608,0,0.656712,-0.700380,0.563109,2.538111,1,0,1,0.0,1.0,0.0
6027,2.067854,1,0.467364,-0.700380,1.024721,0.810498,0,1,0,0.0,1.0,0.0
6729,-2.019706,0,1.035408,1.725478,0.265545,-0.917116,1,0,1,0.0,1.0,0.0
9132,-0.151402,0,-0.574048,-0.007278,-1.219231,0.810498,1,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8499,-1.565533,1,-0.384701,-1.046931,0.730220,0.810498,0,1,0,0.0,1.0,0.0
2064,1.190474,1,0.467364,0.339273,-1.219231,0.810498,1,1,0,1.0,0.0,0.0
7985,0.478247,0,-1.142092,-1.046931,0.317232,0.810498,1,1,0,0.0,1.0,0.0
2303,0.664046,1,0.562038,-1.740033,0.773781,-0.917116,0,1,0,0.0,0.0,1.0


In [61]:
y_train

2724    129697.99
5057    159576.75
6027    186285.52
6729    154344.00
9132    122949.71
          ...    
8499    188343.05
2064    159899.97
7985      5983.70
2303    187194.15
3392    174777.98
Name: EstimatedSalary, Length: 7000, dtype: float64

In [62]:
X_test[scale_columns] = scaler_reg.transform(X_test[scale_columns])
X_test.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
9415,1.221440,0,0.372691,-1.046931,0.846833,0.810498,0,0,0,0.0,1.0,0.0
6377,0.436959,1,-0.574048,1.378927,-1.219231,-0.917116,1,0,1,1.0,0.0,0.0
8019,0.344060,0,3.212907,1.032376,0.982183,0.810498,1,1,0,1.0,0.0,0.0
7754,-0.894595,0,0.278017,0.339273,0.364013,0.810498,1,0,1,0.0,0.0,1.0
4961,0.405993,1,0.562038,-1.740033,0.861327,0.810498,1,0,0,0.0,1.0,0.0


In [41]:
with open('scaler_reg.pkl','wb') as file:
    pickle.dump(scaler_reg,file)

## ANN Implementation

In [63]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [64]:
model = Sequential([
    Dense(64,activation='relu',input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

c:\Vipina\FindJ\Resume\GitHub\ANN_Churn_Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [65]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
opt = tf.keras.optimizers.Adam()
los = tf.keras.losses.MeanAbsoluteError()

In [67]:
# complie model
model.compile(optimizer=opt, loss = los, metrics=['mae'])

In [68]:
# Set up tensorboard callback
log_dir = 'regression/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [69]:
# set up early stopping
earlystopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [70]:
# train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, earlystopping_callback]
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98534.8438 - mae: 98534.8438 - val_loss: 101578.6328 - val_mae: 101578.6328
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98717.2266 - mae: 98717.2266 - val_loss: 99079.7891 - val_mae: 99079.7891
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96046.7656 - mae: 96046.7656 - val_loss: 92689.1719 - val_mae: 92689.1719
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 88060.8203 - mae: 88060.8203 - val_loss: 82659.3281 - val_mae: 82659.3281
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 77713.0234 - mae: 77713.0234 - val_loss: 71164.2891 - val_mae: 71164.2891
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 66383.8125 - mae: 66383.8125 - val_loss: 61552.8164 - val_mae: 61552.8164
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 58847.6289 - mae: 58847.6289 - val_loss: 56102.6523 - val_mae: 56102.6523
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [71]:
model.save('regression.h5')

In [72]:
## Set up a tensorborad and launch it
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [74]:
%tensorboard --logdir regression/fit20241202-004425

Reusing TensorBoard on port 6007 (pid 8316), started 0:00:28 ago. (Use '!kill 8316' to kill it.)